In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

sns.set()

# Graphics in SVG format are more sharp and legible
#%config InlineBackend.figure_format = 'svg'

### code 

In [2]:
def init_data(file_path):
    max_citi = 3600 * 24 * 3
    max_fraud_ctit = 30
    df = pd.read_csv(file_path)
    # print('列参数 = {}'.format(df.columns.values))

    df['fraud_payout'] = (df['payout'] * df['is_fraud']).round(decimals=2)
    df['ctit'] = ((pd.to_datetime(df['postback_datetime']) - pd.to_datetime(df['click_datetime'])) / np.timedelta64(1, 's')).astype(int)
    df['ctit'] = df['ctit'].where(df['ctit'] <= max_citi, other=max_citi)
    df['is_ctit'] = (df['ctit'] <= max_fraud_ctit).astype(np.bool).astype(np.int0)
    df = df.drop('clickid', axis=1)
    df = df.drop('sub_campid', axis=1)
    return df

In [3]:
def _summary_data(dd, need_cumsum=True):
    dd['payout'] = dd['payout'].round(decimals=1)
    dd['fraud_payout_p'] = (dd['fraud_payout'] / dd['payout']).round(decimals=2)
    dd['is_fraud_p'] = (dd['is_fraud'] / dd['is_valid_pb']).round(decimals=2)
    dd['is_ctit_p'] = (dd['is_ctit'] / dd['is_valid_pb']).round(decimals=2)
    
    if need_cumsum:
        cumsum_payout = dd['payout'].cumsum()
        cumsum_is_fraud = dd['is_fraud'].cumsum()
        cumsum_is_valid_pb = dd['is_valid_pb'].cumsum()
        cumsum_fraud_payout = dd['fraud_payout'].cumsum()
        cumsum_is_ctit = dd['is_ctit'].cumsum()
        dd['cumsum_fraud_payout_p'] = (cumsum_fraud_payout / cumsum_payout).round(decimals=2)
        dd['cumsum_is_fraud_p'] = (cumsum_is_fraud / cumsum_is_valid_pb).round(decimals=2)
        dd['cumsum_is_ctit_p'] = (cumsum_is_ctit / cumsum_is_valid_pb).round(decimals=2)
    return dd


def get_summary_by_date(df):
    dd = df.groupby(['date'])['payout', 'is_valid_pb', 'fraud_payout', 'is_fraud', 'is_ctit'].aggregate(np.sum)
    return _summary_data(dd)


def get_tops(df, group_by=['pubid']):
    dd = df.groupby(group_by)['payout', 'is_valid_pb', 'fraud_payout', 'is_fraud', 'is_ctit'].aggregate(np.sum)
    dd = _summary_data(dd, False)
    return dd


### init load

In [4]:
file_path = './data/analy_2018-10-28_28_246.csv.gz'
file_path = './data/analy_2018-10-28_28_.csv.gz'
#file_path = './data/analy_2018-09-30_19_246.csv.gz'
org_df = init_data(file_path)
org_df.head()

,payout,pubid,postback_datetime,publisher_payout,click_ip,is_valid_pb,campid,subid,source,is_fraud,pubpb_filter,fraud_note,date,geo,click_datetime,fraud_payout,ctit,is_ctit
0,0.5814,10294,2018-10-28T00:04:01Z,0.5814,112.215.200.122,True,58966896,1525,81,0,900,ok,2018-10-28,ID,2018-10-27T23:52:07Z,0.0,714,0
1,1.3000,10572,2018-10-28T00:04:05Z,1.6900,174.194.12.96,True,56164481,32816,233,0,0,ok,2018-10-28,US,2018-10-28T00:02:17Z,0.0,108,0
2,0.2000,10236,2018-10-28T00:04:03Z,0.2400,115.178.253.97,True,59589430,544537,150,1,0,blacklist,2018-10-28,ID,2018-10-28T00:03:46Z,0.2,17,1
3,0.6000,10250,2018-10-28T00:04:04Z,0.6000,63.78.215.163,True,49245488,1525,279,1,900,blacklist,2018-10-28,US,2018-10-28T00:00:24Z,0.6,220,0
4,0.7000,10541,2018-10-28T00:04:05Z,1.0500,120.188.64.14,True,48570647,aa5204341f2a50747dadf1c80fa6fd92,162,1,0,blacklist,2018-10-28,ID,2018-10-27T23:54:49Z,0.7,556,0


### 计算每天的fraud百分比平均值

In [5]:
df_summary = get_summary_by_date(org_df)
df_summary

,payout,is_valid_pb,fraud_payout,is_fraud,is_ctit,fraud_payout_p,is_fraud_p,is_ctit_p,cumsum_fraud_payout_p,cumsum_is_fraud_p,cumsum_is_ctit_p
date,,,,,,,,,,,
2018-10-01,128657.4,170643.0,17134.53,22696,1709,0.13,0.13,0.01,0.13,0.13,0.01
2018-10-02,129800.2,170021.0,17169.89,23128,1812,0.13,0.14,0.01,0.13,0.13,0.01
2018-10-03,129614.1,161756.0,17454.91,22780,1535,0.13,0.14,0.01,0.13,0.14,0.01
2018-10-04,116117.9,153113.0,16238.02,22232,1457,0.14,0.15,0.01,0.13,0.14,0.01
2018-10-05,135187.5,175365.0,20528.11,25927,1849,0.15,0.15,0.01,0.14,0.14,0.01
2018-10-06,129695.0,164078.0,18958.58,24378,1517,0.15,0.15,0.01,0.14,0.14,0.01
2018-10-07,135451.8,171002.0,19392.12,25749,1724,0.14,0.15,0.01,0.14,0.14,0.01
2018-10-08,135843.4,171132.0,20648.22,26228,1147,0.15,0.15,0.01,0.14,0.14,0.01
2018-10-09,135046.7,172516.0,19865.32,27224,1140,0.15,0.16,0.01,0.14,0.15,0.01


### 按'date+campid'分组，每天payout >= 100或is_valid_pb >= 100的campid

In [6]:
dd = get_tops(org_df, group_by=['date', 'campid'])

query_list = []
for (date, series) in df_summary.iterrows():
    query_list.append('(date == "{}" and (fraud_payout_p > {} or is_fraud_p > {} or is_ctit_p > {}))'.format(date, series.fraud_payout_p, series.is_fraud_p, series.is_ctit_p))
dd = dd.query(' or '.join(query_list))

dd = dd.reset_index()
dd = dd.sort_values(by=['date','payout'], ascending=[True,False]).query('is_fraud > 0 and (payout >= 100 or is_valid_pb >= 100)')
top_campid_list = dd['campid'].unique()
print('campid {} = {}'.format(len(top_campid_list), top_campid_list))


campid 1054 = [36221642 55668799 48886806 ... 59415447 57165867 51825377]


#### 得到top_campid_list的数据

In [7]:
top_campid_df = org_df[org_df['campid'].isin(top_campid_list)]
top_campid_df.head()

,payout,pubid,postback_datetime,publisher_payout,click_ip,is_valid_pb,campid,subid,source,is_fraud,pubpb_filter,fraud_note,date,geo,click_datetime,fraud_payout,ctit,is_ctit
1,1.30,10572,2018-10-28T00:04:05Z,1.69,174.194.12.96,True,56164481,32816,233,0,0,ok,2018-10-28,US,2018-10-28T00:02:17Z,0.0,108,0
4,0.70,10541,2018-10-28T00:04:05Z,1.05,120.188.64.14,True,48570647,aa5204341f2a50747dadf1c80fa6fd92,162,1,0,blacklist,2018-10-28,ID,2018-10-27T23:54:49Z,0.7,556,0
8,0.29,10569,2018-10-28T00:04:18Z,0.29,182.232.170.82,True,58748046,8_71_p1472_,72,0,900,ok,2018-10-28,TH,2018-10-27T14:03:27Z,0.0,36051,0
9,0.10,10488,2018-10-28T00:04:24Z,0.15,13.124.32.104,True,39887614,100616,175,0,900,ok,2018-10-28,KR,2018-10-28T00:02:38Z,0.0,106,0
11,0.30,10572,2018-10-28T00:04:25Z,0.30,131.0.218.181,True,59668951,32816,356,0,0,ok,2018-10-28,BR,2018-10-28T00:03:29Z,0.0,56,0


#### 大于平均值的campid+pubid，按fraud_payout_p排序

In [16]:
def top_fraud_payout_p(df):
    return df.sort_values(by=['fraud_payout_p', 'is_fraud_p'], ascending=False)

multiple = 2
mean = df_summary.mean()
print('fraud_payout_p={:.2f}，is_fraud_p={:.2f}，is_ctit_p={:.2f}'.format(mean.fraud_payout_p, mean.is_fraud_p, mean.is_ctit_p))

dd = get_tops(top_campid_df, group_by=['campid', 'pubid'])
dd = dd.query('fraud_payout_p > {} or is_fraud_p > {} or is_ctit_p > {}'.format(mean.fraud_payout_p*multiple, mean.is_fraud_p*multiple, mean.is_ctit_p*multiple))
dd = dd.reset_index()
dd = dd.groupby(['campid']).apply(top_fraud_payout_p)
print('shape={}'.format(dd.shape))
dd
#top_campid_pubid_list = dd['pubid'].unique()
#print('top_campid_pubid_list {} = {}'.format(len(top_campid_pubid_list), top_campid_pubid_list))

fraud_payout_p=0.10，is_fraud_p=0.11，is_ctit_p=0.01
shape=(4910, 10)


campid  pubid  payout  is_valid_pb  fraud_payout  is_fraud  \
campid                                                                        
7273137  1      7273137  10436     3.4          7.0          1.44         3   
         3      7273137  10550   128.6        268.0         38.40        80   
         2      7273137  10460    41.8         87.0          9.60        20   
         4      7273137  10572    54.2        113.0         12.48        26   
         0      7273137  10219   305.8        637.0         66.72       139   
15273972 8     15273972  10419     1.9          3.0          0.64         1   
         5     15273972  10236  1029.9       1630.0        283.96       449   
         9     15273972  10491    11.4         18.0          3.20         5   
         7     15273972  10317   975.3       1543.0        254.40       403   
         6     15273972  10291   156.5        245.0         33.24        52   
20805861 10    20805861  10491     1.1          6.0          0.38         2   
22708950 11    22708950  10256     0.5          1.0          0.50         1   
         12    22708950  10266     0.5          1.0          0.50         1   
         14    22708950  10418     0.5          1.0          0.50         1   
         13    22708950  10367    12.0         24.0          2.50         5   
24739074 16    24739074  10291    29.1         51.0         10.26        18   
         18    24739074  10305   203.5        357.0         49.59        87   
         15    24739074  10224    37.6         66.0          8.55        15   
         17    24739074  10294   201.2        347.0         44.46        78   
         19    24739074  10317    55.3         97.0         11.40        20   
26407989 20    26407989  10296  1604.1       8824.0        423.92      2324   
26981063 21    26981063  10316     4.8         17.0          0.84         3   
         22    26981063  10555    28.6        102.0          5.04        18   
29089198 25    29089198  10291   166.4        128.0          2.60         2   
         29    29089198  10418   141.7        109.0          2.60         2   
         23    29089198  10224  1701.7       1309.0         14.30        11   
         24    29089198  10250    18.2         14.0          0.00         0   
         26    29089198  10294    10.4          8.0          0.00         0   
         27    29089198  10305    10.4          8.0          0.00         0   
         28    29089198  10397     1.3          1.0          0.00         0   
...                 ...    ...     ...          ...           ...       ...   
59719286 4879  59719286  10310   108.9        363.0         29.10        97   
         4880  59719286  10316     4.5         15.0          1.20         4   
59733620 4882  59733620  10437     2.9          3.0          1.94         2   
         4884  59733620  10524   125.8        130.0         48.50        50   
         4883  59733620  10465   184.9        191.0         54.32        56   
59745767 4887  59745767  10360     0.4          1.0          0.35         1   
         4889  59745767  10454     2.4          7.0          1.40         4   
         4888  59745767  10436    15.4         44.0          5.25        15   
         4890  59745767  10460     8.0         23.0          2.10         6   
         4886  59745767  10316     7.0         20.0          1.75         5   
         4885  59745767  10305     4.6         13.0          1.05         3   
59750750 4894  59750750  10317     1.8          4.0          0.92         2   
         4893  59750750  10266     2.3          5.0          0.92         2   
         4892  59750750  10236    65.8        143.0         21.62        47   
         4891  59750750  10217    10.6         23.0          3.22         7   
         4895  59750750  10397   676.7       1471.0        203.32       442   
         4896  59750750  10408    60.3        131.0         17.02        37   
         4897  59750750  10436     2.3          5.0          0.00  

In [ ]:
dd = get_tops(top_campid_df, group_by=['campid', 'pubid', 'subid'])
dd = dd.query('fraud_payout_p > {} or is_fraud_p > {} or is_ctit_p > {}'.format(mean.fraud_payout_p*multiple, mean.is_fraud_p*multiple, mean.is_ctit_p*multiple))
dd = dd.reset_index()
dd = dd.groupby(['campid', 'pubid']).apply(top_a)
print('shape={}'.format(dd.shape))
dd

### 按'date+pubid'分组，每天payout >= 100的pubid

In [9]:
dd = get_tops(org_df, df_summary, group_by=['date', 'pubid'])
dd = dd.reset_index()
dd = dd.sort_values(by=['date', 'payout'], ascending=[True, False]).query('payout >= 100')
top_pubid_list = dd['pubid'].unique()
print('pubid {} = {}'.format(len(top_pubid_list), top_pubid_list))

TypeError: get_tops() got multiple values for argument 'group_by'

In [ ]:
mix_pubid = list(set(top_campid_pubid_list.tolist()+top_pubid_list.tolist()))
print('mix_pubid {} = {}'.format(len(mix_pubid), mix_pubid))

### 按'date+source+campid'分组，前15的campid

In [ ]:
def top(df, n=5, column='payout'):
    return df.sort_values(by=column, ascending=False)[:n]

dd = get_tops(org_df, df_summary, group_by=['date', 'source', 'campid'])
dd = dd.reset_index()
dd = dd.groupby(['date', 'source']).apply(top, n=15, column='payout')
#dd = dd['payout'].groupby(level=['date', 'source'], group_keys=False).nlargest(15)

top_source_campid = dd['campid'].unique()
print('source campid {} = {}'.format(len(top_source_campid), top_source_campid))
dd

In [ ]:
dd = get_tops(org_df, df_summary, group_by=['date', 'pubid', 'campid'])
#dd = dd.sort_values(by=['date', 'source', 'payout'], ascending=[True, True, False]).query('payout >= 100')
dd = dd['payout'].groupby(level=['date', 'pubid'], group_keys=False).nlargest(15)
dd = dd.reset_index()
top_pubid_campid = dd['campid'].unique()
print('pubid campid {} = {}'.format(len(top_pubid_campid), top_pubid_campid))

In [ ]:
top_mix_campid = list(set(top_source_campid.tolist()+top_pubid_campid.tolist()))
print('mix campid {} = {}'.format(len(top_mix_campid), top_mix_campid))